In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import os 
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family']='Arial'

- `extend`
    - `avg`~...
        - w/ conveyor
        - w/o conveyor
    - `attrition`~...
        - w/ conveyor
        - w/o conveyor
- `charge`
    - `avg`~...
        - w/ conveyor
        - w/o conveyor
    - `attrition`~...
        - w/ conveyor
        - w/o conveyor

In [102]:
# funcs def.
def showSig(x):
    x=round(x,4)
    if x<0.001:
        return(f'**** {x}')
    if x<0.01:
        return(f'*** {x}')
    if x<0.05:
        return(f'** {x}')
    if x<0.1:
        return(f'* {x}')
    else:
        return(x)
    
def modelSumm(model, verbose=False, sig=False):
    if verbose==True:
        return model.summary()
    if sig==True:
        return pd.DataFrame({ 'p_value': model.pvalues.apply(showSig), 'std_error': model.bse.apply(lambda x: round(x,4)) })
    return pd.DataFrame({ 'p_value': model.pvalues.apply(lambda x: round(x,4)), 'std_error': model.bse.apply(lambda x: round(x,4))})

In [136]:
extend=pd.read_csv('wages_extend_month.csv')
extend['batchch']=extend['numbatches'].diff()
charge=pd.read_csv('wages_charge_month.csv')
charge['batchch']=charge['numbatches'].diff()
extend['pschange']=extend.loc[extend['pschange']<140, 'pschange'].replace(np.inf, 1000)
charge['pschange']=charge.loc[charge['pschange']<140, 'pschange'].replace(np.inf, 1000)
ind_vars=["avgprice","cus1price","mbpricing","mbpay","numbuyers","pschange","workers","allps","conveyor"]

In [84]:
# handle inf:
infRec=[]
for i,r in extend.iterrows():
    if np.isinf(r['pschange']):
        infRec.append(i)

extend.drop(infRec, inplace=True)

In [141]:
# ext:avg~..+conveyor
ext_avg_wc=smf.ols(f'avg_pay ~ avgprice + cus1price + mbpricing + mbpay + numbuyers+ pschange+ stylesch + numstyles + conveyor',
            data=extend).fit()

modelSumm(ext_avg_wc, sig=0)

,p_value,std_error
Intercept,0.0117,2242.8111
conveyor[T.A-2],0.0611,254.4245
conveyor[T.A-3],0.4074,271.5844
conveyor[T.A-4],0.1468,236.1442
conveyor[T.B-1],0.8332,249.1510
conveyor[T.B-2],0.9532,470.1734
conveyor[T.B-3],0.2108,293.5894
conveyor[T.B-4],0.1756,295.0171
conveyor[T.C-1],0.0084,341.2940
conveyor[T.C-2],0.0011,276.0804


0.0117(2242.8111)
0.1753(56.4222)
0.2870(62.3347)
0.1304(575.5283)
0.0722(320.3264)
0.8945(51.2876)
0.0467(0.9900)
0.4806(4.4769)
0.3432(6.5527)
0.0611(254.4245)
0.4074(271.5844)
0.1468(236.1442)
0.8332(249.1510)
0.9532(470.1734)
0.2108(293.5894)
0.1756(295.0171)
0.0084(341.2940)
0.0011(276.0804)
0.0103(284.8906)
0.0027(284.6851)
0.0000(294.9789)
0.0000(362.6410)
0.0000(326.1873)

In [131]:
ext_avg_woc=smf.ols('avg_pay ~ avgprice + cus1price + mbpricing + mbpay + numbuyers + pschange +stylesch + numstyles+conveyor',
            data=extend).fit()
modelSumm(ext_avg_woc, verbose=1)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                avg_pay   R-squared:                       0.721
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     8.209
Date:                Tue, 01 Oct 2024   Prob (F-statistic):           5.55e-12
Time:                        21:22:35   Log-Likelihood:                -662.91
No. Observations:                  93   AIC:                             1372.
Df Residuals:                      70   BIC:                             1430.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        5804.4638   2242.811      2.588      0.012    1331.318    1.03e+04
conveyor[T.A-2]  -484.3600    254.425     -1.904      0.061    -991.794      23.074
conveyor[T.A-3]  -226.3496    271.584     -0.833      0.407    -768.008     315.308
conveyor[T.A-4]   346.4601    236.144      1.467      0.147    -124.515     817.435
conveyor[T.B-1]    52.6578    249.151      0.211      0.833    -444.258     549.574
conveyor[T.B-2]    27.6762    470.173      0.059      0.953    -910.055     965.407
conveyor[T.B-3]  -370.8078    293.589     -1.263      0.211    -956.353     214.738
conveyor[T.B-4]  -403.6965    295.017     -1.368      0.176    -992.090     184.697
conveyor[T.C-1]  -925.9275    341.294     -2.713      0.008   -1606.617    -245.238
conveyor[T.C-2]  -938.0650    276.080     -3.398      0.001   -1488.690    -387.440
conveyor[T.C-3]  -751.5295    284.891     -2.638      0.010   -1319.726    -183.333
conveyor[T.C-4]  -884.4526    284.685     -3.107      0.003   -1452.239    -316.666
conveyor[T.C-5] -1434.2162    294.979     -4.862      0.000   -2022.533    -845.899
conveyor[T.C-6] -2648.8766    362.641     -7.304      0.000   -3372.141   -1925.612
conveyor[T.C-7] -1571.5267    326.187     -4.818      0.000   -2222.087    -920.967
avgprice          -77.2595     56.422     -1.369      0.175    -189.790      35.271
cus1price          66.8853     62.335      1.073      0.287     -57.437     191.208
mbpricing         880.8402    575.528      1.530      0.130    -267.015    2028.695
mbpay             584.6864    320.326      1.825      0.072     -54.185    1223.557
numbuyers           6.8237     51.288      0.133      0.895     -95.466     109.114
pschange            2.0044      0.990      2.025      0.047       0.030       3.979
stylesch           -3.1749      4.477     -0.709      0.481     -12.104       5.754
numstyles           6.2528      6.553      0.954      0.343      -6.816      19.322
==============================================================================
Omnibus:                        1.476   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.478   Jarque-Bera (JB):                0.946
Skew:                           0.022   Prob(JB):                        0.623
Kurtosis:                       3.492   Cond. No.                     3.26e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [140]:
ext_avg_woc=smf.ols('avg_pay ~ avgprice + cus1price + mbpricing + mbpay + numbuyers + pschange +stylesch + numstyles+conveyor',
            data=charge).fit()
modelSumm(ext_avg_woc, verbose=1)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                avg_pay   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     7.473
Date:                Tue, 01 Oct 2024   Prob (F-statistic):           4.93e-11
Time:                        22:03:29   Log-Likelihood:                -671.38
No. Observations:                  94   AIC:                             1387.
Df Residuals:                      72   BIC:                             1443.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        8124.9103   2147.086      3.784      0.000    3844.773    1.24e+04
conveyor[T.A-2]  -628.9809    237.677     -2.646      0.010   -1102.782    -155.180
conveyor[T.A-3]  -325.6297    271.347     -1.200      0.234    -866.551     215.291
conveyor[T.A-4]   361.5979    217.212      1.665      0.100     -71.406     794.602
conveyor[T.B-1]    61.5007    265.515      0.232      0.817    -467.794     590.796
conveyor[T.B-2] -1.232e-11   4.25e-12     -2.899      0.005   -2.08e-11   -3.85e-12
conveyor[T.B-3]  -308.9341    280.993     -1.099      0.275    -869.084     251.216
conveyor[T.B-4]  -337.5910    307.304     -1.099      0.276    -950.190     275.008
conveyor[T.C-1]  -343.0151    355.046     -0.966      0.337   -1050.786     364.756
conveyor[T.C-2]  -599.5360    279.484     -2.145      0.035   -1156.678     -42.394
conveyor[T.C-3]  -446.6894    285.955     -1.562      0.123   -1016.731     123.352
conveyor[T.C-4]  -568.1660    288.263     -1.971      0.053   -1142.807       6.475
conveyor[T.C-5] -1168.7563    293.009     -3.989      0.000   -1752.858    -584.654
conveyor[T.C-6] -1865.5824    349.930     -5.331      0.000   -2563.155   -1168.010
conveyor[T.C-7] -1463.8177    398.118     -3.677      0.000   -2257.452    -670.183
avgprice          -31.1071     44.201     -0.704      0.484    -119.221      57.007
cus1price          56.9044     51.725      1.100      0.275     -46.207     160.016
mbpricing          78.1274    512.767      0.152      0.879    -944.055    1100.310
mbpay             235.0306    312.717      0.752      0.455    -388.360     858.421
numbuyers         -44.6901     57.575     -0.776      0.440    -159.465      70.084
pschange            1.9929      1.110      1.796      0.077      -0.219       4.205
stylesch           -6.6691      7.135     -0.935      0.353     -20.893       7.554
numstyles           2.2658      9.739      0.233      0.817     -17.150      21.681
==============================================================================
Omnibus:                        1.036   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.596   Jarque-Bera (JB):                0.582
Skew:                          -0.154   Prob(JB):                        0.747
Kurtosis:                       3.231   Cond. No.                     9.42e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.43e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""